Advanced - Structural Learning <br>
Mixture of expert and data

In [ ]:
install.packages(
  "bnlearn",
  repos = "https://cloud.r-project.org",   # CRAN mirror
  dependencies = TRUE                      # pulls in igraph, gRain, etc.
)

In [ ]:
install.packages(
  "gRain",
  repos = "https://cloud.r-project.org",   # CRAN mirror
  dependencies = TRUE                      # pulls in igraph, gRain, etc.
)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cowplot’, ‘Deriv’, ‘doBy’, ‘litedown’, ‘gRim’, ‘markdown’, ‘microbenchmark’


Loading required package: gRbase


Attaching package: ‘gRbase’


The following objects are masked from ‘package:bnlearn’:

    ancestors, children, nodes, parents




In [112]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("Rgraphviz")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'BiocVersion', 'Rgraphviz'

also installing the dependencies ‘BiocGenerics’, ‘graph’


Old packages: 'data.table', 'purrr', 'waldo'



In [113]:
install.packages(
  "lattice",
  repos = "https://cloud.r-project.org",   # CRAN mirror
  dependencies = TRUE                      # pulls in igraph, gRain, etc.
)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘deldir’, ‘png’, ‘jpeg’, ‘interp’, ‘latticeExtra’, ‘colorspace’




In [149]:
install.packages(
  "penalized",
  repos = "https://cloud.r-project.org",   # CRAN mirror
  dependencies = TRUE                      # pulls in igraph, gRain, etc.
)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“dependency ‘globaltest’ is not available”


In [169]:
library(bnlearn)
library(gRain)
library(penalized)

library(lattice)
library(Rgraphviz)
library(rbmn)

In [ ]:
# Gaussian Network
net = model2network("[A][B][C|A:B]")

# Case 1
# Expert Defined - local distributions
distA = list(coef = c("(Intercept)" = 2), sd = 1)
distB = list(coef = c("(Intercept)" = 1), sd = 1.5)
distC = list(coef = c("(Intercept)" = 0.5, "A" = 0.75, "B" = 1.32), sd = 0.4)

cfit = custom.fit(net, dist = list(A = distA, B = distB, C = distC))

#graphviz.plot(cfit)

# Case 2 - data driven
# Suppose - we want to fit "all" of the local distributions using data
data(gaussian.test)
#head(gaussian.test)

#?lm
# use lm to estimate the local regressions
distA = lm(A ~ 1, data = gaussian.test)
distB = lm(B ~ 1, data = gaussian.test)
distC = lm(C~ A + B, data = gaussian.test)

# take the new local paramaterizations and push them into the bn
cfit =  custom.fit(net, dist = list(A = distA, B = distB, C = distC))
cfit$A

# Case 3 - a mixture
# Suppose - we want to fit "some" of the local distributions using data
distA = list(coef = c("(Intercept)" = 2), sd = 1) #expert
distB = list(coef = c("(Intercept)" = 1), sd = 1.5) #expert
distC = lm(C~ A + B, data = gaussian.test) # data driven
#cfit =  custom.fit(net, dist = list(A = distA, B = distB, C = distC))
#cfit$A
#cfit$C

# Shrinkage
distC = penalized(C~A+B, data = gaussian.test, lambda1 = 0, lambda2 = .5, trace = FALSE)
#distC
cfit =  custom.fit(net, dist = list(A = distA, B = distB, C = distC))


  Parameters of node A (Gaussian distribution)

Conditional density: A
Coefficients:
(Intercept)  
   1.007493  
Standard deviation of the residuals: 1.004233 

Warning message in full.spec.backend(fitted, dist):
“different nodes have different number of residuals or fitted values, disregarding.”


In [ ]:
# Multinomial BN
data(learning.test)
#head(learning.test)

dag = model2network("[A][C][F][B|A][D|A:C][E|B:F]")
#graphviz.plot(dag)

# Case 1
# using the data to estimate the local distribution
my.bn = bn.fit(dag, data = learning.test)
my.bn$C

# Case 2  - mixture of expert and data
# new "information" from experts -- want to specify some of the local distributions
cpt = coef(my.bn$C)
cpt[1:3] = c(0.50, 0.25, 0.25)
my.bn$C = cpt
my.bn$C



  Parameters of node C (multinomial distribution)

Conditional probability table:
      a      b      c 
0.7434 0.2048 0.0518 


  Parameters of node C (multinomial distribution)

Conditional probability table:
    a    b    c 
0.50 0.25 0.25 